# Carnegie Hall Trends Data Querying, Cleanup and Graphing

### By Vivek Mittal-Henkle and Owen Yaggy
#### December 21, 2023

### A note on dependencies

This project relies on several data files, .csv and .pkl, to run all of its code. Contained within this repository are all the
files necessary such that any user of this notebook will not have to make any SPARQL queries to obtain the necessary data.
However, in some cases we show the code that transforms the data resulting from a SPARQL query into a new CSV. We do not
include the CSVs resulting directly from the SPARQL queries in this repository, only the resultant CSVs that are necessary
for this project to run its core features.

# [RUN] Import statements

In [2]:
import pandas as pd
import plotly.express as px
import os
import requests
import csv

## A query with optional results ##
This allows the query to return a result even if it has certain attributes missing.
This returns all the data we're looking for.

```PREFIX schema: <http://schema.org/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX mo: <http://purl.org/ontology/mo/>

SELECT *
WHERE {
	?event a schema:Event .
  	OPTIONAL {
    	?event schema:startDate ?date .
  	}
    OPTIONAL { ?event rdfs:label ?title . }
  	OPTIONAL {
    	?event mo:genre ?genre .
  		FILTER contains(str(?genre), "http://data.carnegiehall.org/genres/")
  		OPTIONAL { ?genre rdfs:label ?genreLabel . }
  	}
    OPTIONAL {
    	?event schema:subEvent/schema:workPerformed ?workperformed .
    	OPTIONAL { ?workperformed rdfs:label ?work . }
  		OPTIONAL {
    		?workperformed dcterms:creator ?composer .
      	OPTIONAL { ?composer schema:name ?composerName . }
  			OPTIONAL {
    			?composer skos:exactMatch ?skos .
    			FILTER contains(str(?skos), "wiki") 
  			}
  		}
  	}
}
LIMIT 10000
OFFSET 0        # this offset should be increased in increments of 10,000 until all data has been queried and downloaded
```

A significant drawback is the enforced results limit of 10,000 on the Carnegie Data Lab website. This necessitates 38 queries offset by increments of 10,000 to query the full suite of data. The resulting data is downloaded as separate CSVs, which are concatenated using the below code.

# [do not run] CSV concatenator

If SPARQL query limits mean that the necessary data is stored in many disparate csv files, this code will take the path to the folder containing all those csvs (and nothing else!!) and the desired file path of the resultant csv that will concatenate all the smaller csvs.

This is a tool used in the process of working with the data. However, since the resultant data has already been stored, it is not necessary to run it.

In [21]:
# code to combine several downloaded sparql query results

def combine_csvs(folder_path, file_name):
    """
    Combines several csvs into one large one
    :param folder_path: path to the folder containing all the csvs to be contained and NOTHING ELSE
    :param file_name: path to the file that will store the combined csv, including .csv at the end!
    :return: 
    """
    
    # record all the files in the folder
    files = os.listdir(folder_path)
    
    # blank list to store dataframes for each individual csv
    df_list = []
    # iterate through smaller csvs, make a dataframe for them
    # and add that dataframe to a list of dataframes
    for csv in files:
        file_path = os.path.join(folder_path, csv)
        csv_df = pd.read_csv(file_path)
        df_list.append(csv_df)
    
    # concatenate all the dataframes
    big_df = pd.concat(df_list, ignore_index=True)
    # save the result in a csv
    big_df.to_csv(file_name)

# [do not run] Function to gather all nationalities

First, make the below query:
```
# Fetch Wikidata ID for all composers who have one
PREFIX marcRel: <http://id.loc.gov/vocabulary/relators/>
PREFIX schema: <http://schema.org/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?composer ?skos WHERE {
	?composer schema:hasOccupation marcRel:cmp . #occupation of composer
  	?composer skos:exactMatch ?skos .
  	filter contains(str(?skos), "wiki")
}
LIMIT 10000
OFFSET 0
```

The result should be fewer than 10,000 composer wikidata IDs. If a query has over 10,000 results, download all the CSVs and combine them using the CSV concatenator above.

Then run the below function with the CSV resulting from the query passed to `path_to_query_results`, and the `path_to_nationalities_csv` parameter set to the path to the file where the nationalities for each composer should be stored, for example `composer_nationalities.csv`.

This is a tool used in the process of working with the data. However, since the resultant data has already been stored, it is not necessary to run it.

In [26]:
def store_nationalities(path_to_query_results_csv, path_to_nationalities_pkl):
    """Creates a PKL listing all Carnegie composer IDs and all their nationalities, which are stored in a list."""
    nationalities_df = pd.read_csv(path_to_query_results_csv)
    nationalities_df.insert(2, 'nationalities', pd.Series(dtype=str))
    
    for index in nationalities_df.index:
            # isolate the data in the 'skos' row
            wikidata_id = nationalities_df.loc[index, 'skos']
            # isolate the unique wikidata item identifier beginning with 'Q'
            wikidata_id = wikidata_id[wikidata_id.index('Q'):]
            
            # API call to wikidata to get the "countries of citizenship" for this item
            request = requests.get(f'https://www.wikidata.org/w/rest.php/wikibase/v0/entities/items/{wikidata_id}/statements?property=P27')
            if len(dict(request.json())) == 0:
                request = requests.get(f'https://www.wikidata.org/w/rest.php/wikibase/v0/entities/items/{wikidata_id}/statements?property=P495')
            
            # create blank list to store nationalities
            nationalities = []
            
            try:
                # iterate through all countries of citizenship for the composer
                for result in dict(request.json())['P27']:
                    # store the nationality ID(s)
                    nationalities.append(result['value']['content'])
            # if the composer doesn't have a nationality
            except KeyError:
                # see if it's actually a musical group with an association to a specific country/countries
                try:
                    for result in dict(request.json())['P495']:
                        # store the country ID(s)
                        nationalities.append(result['value']['content'])
                # if no associated countries found, add the value for "statelessness")
                except KeyError:
                    nationalities.append('Q223050')         # stateless
            
            # iterate through all the nationality IDs found
            for i in range(len(nationalities)):
                # convert the ID to the label for that country
                nationalities[i] = requests.get(f'https://www.wikidata.org/w/rest.php/wikibase/v0/entities/items/{nationalities[i]}/labels/en').json()
            
            # store the result in the dataframe
            nationalities_df.at[index, 'nationalities'] = nationalities
    
    # iterate through the nationalities
    #for index in nationalities_df.index:
    #    # convert to a string
    #    nationalities_df.at[index, 'nationalities'] = nationalities_df.at[index, 'nationalities'].to_string()
    
    # Create and store a PKL based on the nationality data
    nationalities_df.to_pickle(path_to_nationalities_pkl)

store_nationalities('path/to/composers_wikidatas.csv', 'Data/composers_nationalities.pkl')

# [do not run] Function to format data

In [2]:
def add_to_event_info(column_name: str, item: str, event_df, base_df, row):
    """
    adds a particular column to the sub-dataframe event_df
    by looking at all the work rows in the base dataframe that are from the same event
    """

    event_df[column_name] = base_df[base_df['event'] == row['event']][item]
    return event_df

def progress_indicator(index):
    """For the two processes that take a very long time to run,
    this code will occasionally print a status update by seeing
    if the current position in the 373,232 line csv matches a
    landmark interval, and if so printing that interval (e.g.
    10%, 90%)"""
    pos = index / 373232
    if 0.99 < pos < 0.9901:
        print('Almost done!')
    elif 0.9 < pos < 0.9001:
        print('90%')
    elif 0.8 < pos < 0.8001:
        print('80%')
    elif 0.7 < pos < 0.7001:
        print('70%')
    elif 0.6 < pos < 0.6001:
        print('60%')
    elif 0.5 < pos < 0.5001:
        print('50%')
    elif 0.4 < pos < 0.4001:
        print('40%')
    elif 0.3 < pos < 0.3001:
        print('30%')
    elif 0.2 < pos < 0.2001:
        print('20%')
    elif 0.1 < pos < 0.1001:
        print('10%')
    elif 0.01 < pos < 0.0101:
        print('1% done!')
    elif 0.001 < pos < 0.0011:
        print('0.1% done!')

def create_event_data(items_to_add, base_df, row, index):
    progress_indicator(index)
    event_data = pd.DataFrame()
    for item in items_to_add:
        event_data = add_to_event_info(item, item, event_data, base_df, row)
    return event_data

def cleaner(x, items_to_add):
    """
    1. takes in an uncleaned dataframe x
    2. creates the sub-dataframe event_data for each event with the information
    pertaining to all the works in that event
    3. removes redundant rows
    4. adds new column for simple four digit year
    5. cleans genreLabel column
    6. returns cleaned dataframe
    """
    
    print('Step 1: Create event data')
    x['event_data'] = [create_event_data(items_to_add, x, row, index) for index, row in x.iterrows()]
    
    print('Step 2: Get rid of duplicate rows')
    events = []
    for index, row in x.iterrows():
        progress_indicator(index)
        # get rid of multiple rows for the same event
        if row['event'] in events:
            x = x.drop(index)
        else:
            events.append(row['event'])
    
    print('Step 3: Adding additional columns')
    # creates a simple year column of type int
    x['year'] = [int(date[:4]) for date in x['date']]
    # cleans the column genreLabel
    x['genreLabel'] = x['genreLabel'].str.lower()
    
    print('DONE')

    return x

# [do not run] Function to add nationalities to dataframe

In [40]:
def add_nationalities(input_df, col=6):
    """Combines nationality data csv and carnegie hall data csv together"""
    # in the main list of event data, nationalities should be inserted in the 6th column
    # however, in the list of all composers, nationality should be inserted in a different column
    input_df.insert(col, "nationalities", pd.Series(dtype=str))
    
    names_with_nationalities = pd.read_pickle('Data/composers_nationalities.pkl')
    
    # Iterate through the dataframe, adding nationalities when possible
    for index in input_df.index:
        nationalities = names_with_nationalities.loc[names_with_nationalities['composer'] == input_df.loc[index, 'composer']]
        nationalities = nationalities['nationalities']
        try:
            nationalities = nationalities.get(nationalities.keys()[0])
        except IndexError:
            nationalities = 'statelessness'
        try:
            input_df.at[index, 'nationalities'] = nationalities
        except KeyError:
            print(input_df.loc[index])
    
    return input_df

# [RUN] Graphing functions

In [14]:
def create_event_frequency_list(df, lookup_range, column, specific_value, normalize=False):
    """
    Given a lookup_range of years, and a specific_value in a column to look for, this function
    returns a list of frequencies of that specific_value per year.
    
    Note that specific_value must appear in column.
    
    Setting the normalize parameter to True will instead return a list of the proportions
    count(specific_value)/total number of performances per year
    """
    
    # converts user inputs into column names in df
    column_key = {'Genre': 'genreLabel', 'Nationality': 'nationalities', 'Work': 'workperformed', 'Composer': 'composer'}
    column = column_key[column]
    
    # cleaning work to make sure everything matches up
    if column == 'genreLabel':
        specific_value = specific_value.lower()
    elif column in ('workperformed', 'composer'):
        specific_value = specific_value[specific_value.index('#') + 1:specific_value.index(')')]
    
    frequency_list = []
    
    
    for year in lookup_range:

        # if it has to do with works and not events (composer, nationality, etc.)
        if column in df['event_data'][0].columns:

            # create smaller dataframe with events only in that year to simplify
            sub_df = df[df['year'] == year].copy()

            # create a boolean column for whether the specific value can be found in a work performed at that event
            has_value = []

            # iterate through events in specific year
            for index, row in sub_df.iterrows():
                # isolate the entry in the column for the current event
                important_column = sub_df['event_data'][index][column]

                # check if the desired value is present in the column entry
                # for nationalities, need to check if any of the desired nations are present in the column
                if column == 'nationalities':
                    # assume not present
                    any_nationality_present = False
                    # iterate through nations
                    for nation in specific_value:
                        # if a matching nation is found
                        if nation in important_column.to_string():
                            # mark that this nationality group is present for this event
                            any_nationality_present = True
                            # stop searching for matches in this event
                            break
                    # add the boolean storing whether this nationality group was present for this event
                    has_value.append(any_nationality_present)
                else:
                    # add the boolean storing whether the desired value was present for this event
                    has_value.append(f"/{specific_value}" in important_column.to_string())
                    

            if column == 'nationalities':
                sub_df[specific_value[0]] = has_value
            else:
                sub_df[specific_value] = has_value

            # create the frequency list
            try:  # if the desired event has occurred in this year, add the number of times it occurred
                if column == 'nationalities':
                    frequency_list.append(sub_df.value_counts(specific_value[0], normalize=normalize).to_dict()[True])
                else:
                    frequency_list.append(sub_df.value_counts(specific_value, normalize=normalize).to_dict()[True])
            except KeyError:  # if the desired event has not occurred in this year
                frequency_list.append(0)

        # if it has to do with events (genre, work, etc)
        else:
            # get a dictionary of the frequencies
            attribute_counts = df[df['year'] == year].value_counts(column, normalize=normalize)

            # getting the count for the specific value
            try:
                if column == 'nationalities':
                    frequency_list.append(attribute_counts[specific_value[0]])
                else:
                    frequency_list.append(attribute_counts[specific_value])
            except KeyError:
                frequency_list.append(0)

    return frequency_list


def make_bar_chart(df, column, specific_value, lookup_range=(0, 0), normalize=False):
    """
    make a bar chart of the frequency of "specific_value", which is a value in "column" over "lookup_range" years
    
    Note: if normalize=True it will instead return a bar chart of proportions
    """
    # Create a DataFrame for bar chart

    # years is the x-axis
    # this first if statement allows you to make the chart for a subset of the years
    if lookup_range != (0, 0):
        years = []
        for year in range(lookup_range[0], lookup_range[1] + 1):
            years.append(year)

    else:
        years = list(set(df['year'].to_list()))


    # list of frequencies
    frequency = create_event_frequency_list(df, years, column, specific_value, normalize)


    bar_data = {'Years': years,
                'frequency': frequency}
    df_bar = pd.DataFrame(bar_data)

    # The barchart with Plotly Express specifying the source df, the columns to use as x and y axes,
    # labels to use for those axes, and an overall title for the figure

    fig = px.bar(df_bar,
                 x = 'Years', y= 'frequency',
                 labels={'Years': 'Years', 'frequency': f'Performances of {column.title()}: {specific_value}'},
                 title=f'Performances of {column.title()}: {specific_value} by Year',
                )
    # Set width and height in pixels
    fig.update_layout(width=600, height=400)
    fig.show()

# [do not run] Store results from the query in a .pkl file

This processes the data queried from the Carnegie SPARQL service and stores the result in a `.pkl` file. Unlike a CSV, a pickle file retains the information contained in a dataframe, such as item types, so that the data can be more easily processed when it is uploaded and used in the Streamlit app.

In [0]:
# read the CSV containing all event data (the concatenation of all the (38!) SPARQL queries)
df = pd.read_csv("events_list.csv")

# the columns that need to be added to our new dataframe for each event
items_to_add = ['workperformed', 'composerName', 'nationalities']

# add the nationalities to the dataframe
df = add_nationalities(df)

The cleaner data processes the data in the way that is easiest for us to read when creating graphs. However, it has an extreme computational cost: to process the 380,000+ rows of data returned by all the SPARQL queries necessary to gather the entire Carnegie Hall dataset, this cleaner function takes just under **12 hours** to run.

In [ ]:
# This line of code is the most computationally intensive, and can take many hours to run.
data = cleaner(df, items_to_add)

In [ ]:
# Store the dataframe resulting from the cleaner into a pickle file, which maintains the dataframe characteristics better than a CSV
data.to_pickle('Data/event_data.pkl')

# [RUN] Demo

Enter your own parameters in `make_bar_chart` for attribute and value.

In [15]:
# Input the exact attribute and value you want to graph
event_data = pd.read_pickle('Data/event_data.pkl')
make_bar_chart(event_data, 'Work', '(#68080)')

68080
- 2005 -
304090    http://data.carnegiehall.org/works/46169
304091    http://data.carnegiehall.org/works/91426
304092    http://data.carnegiehall.org/works/44234
304093    http://data.carnegiehall.org/works/68079
304094    http://data.carnegiehall.org/works/45814
304095    http://data.carnegiehall.org/works/68080
304096       http://data.carnegiehall.org/works/68
304097    http://data.carnegiehall.org/works/68086
304098    http://data.carnegiehall.org/works/68085
304099    http://data.carnegiehall.org/works/68084
304100    http://data.carnegiehall.org/works/55702
304101    http://data.carnegiehall.org/works/42288
304102     http://data.carnegiehall.org/works/2617
304103     http://data.carnegiehall.org/works/9760
304104    http://data.carnegiehall.org/works/31014
304105    http://data.carnegiehall.org/works/31899
304106    http://data.carnegiehall.org/works/68081
304107    http://data.carnegiehall.org/works/25688
304108    http://data.carnegiehall.org/works/15727
304109    http:/

# Tools for Streamlit

In the process of the making the Streamlit app, we needed to create lists of all the genres, all the nationalities, all the works, and all the composers
that have (been) performed at Carnegie Hall. We made queries for each of these lists, and created tools to clean the data.

### Genre

We collected and downloaded a CSV with all genres by making the following query:
```
PREFIX schema: <http://schema.org/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX mo: <http://purl.org/ontology/mo/>

SELECT DISTINCT ?genreLabel WHERE {
	?event a schema:Event ;
  		mo:genre ?genre .
  ?genre rdfs:label ?genreLabel .
}
```

The following code creates a new csv that adds a column header, 'Genre', and capitalizes all the genres. It first reads a CSV resulting from the above query.

In [11]:
df = pd.read_csv('path/to/genre_query.csv')

with open('genreLabels_list.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(['Genre'])
    for genreLabel in df['genreLabel']:
        spamwriter.writerow([genreLabel.title()])

### Nationality

From running `store_nationalities` from earlier, we have a PKL containing all composers and their nationalities. We can use this to create a list of every
nationality held by a composer in the Carnegie Hall database, thus creating a list of options for users in the Streamlit app selecting nationality.

In [89]:
df = pd.read_pickle('Data/composers_nationalities.pkl')

all_nations = set()
last = None
for item in df['nationalities']:
    for nation in item:
        all_nations.add(nation)\

all_nations = sorted(all_nations)

In [96]:
# store the results in a csv
with open('nationalities_list.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='"')
    spamwriter.writerow(['Nation'])
    for item in all_nations:
        spamwriter.writerow([item])

### Works

There are just over 100,000 unique works in the Carnegie Hall database. So we have to make queries of 10,000 at a time, incrementing the offset by 10,000,
to create 11 CSVs. Those then need to be combined using the CSV concatenator. This combined CSV is all that is necessary to create a list of works,
combined with their composer and unique identifier. The below query is used, and fetches all the necessary information:

```
#Find works
PREFIX schema: <http://schema.org/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX mo: <http://purl.org/ontology/mo/>
SELECT * WHERE {
	?work a schema:MusicComposition .
    OPTIONAL {
    	?work rdfs:label ?title .
        OPTIONAL {
      		?work dcterms:creator ?composer .
      		OPTIONAL { ?composer rdfs:label ?composerLabel . }
    	}
  	} 
}
LIMIT 10000
OFFSET 0
```

### Composers

To find all the composers, the following query is made:
```
# Fetch all composers
PREFIX marcRel: <http://id.loc.gov/vocabulary/relators/>
PREFIX schema: <http://schema.org/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?composer ?composerLabel WHERE {
	?composer schema:hasOccupation marcRel:cmp . #occupation of composer
  	OPTIONAL { ?composer rdfs:label ?composerLabel . }
}
LIMIT 10000
OFFSET 0
```
There are just over 20,000 composers, necessitating 3 queries to fetch them all. The CSVs created from each query should be combined using the CSV concatenator.

### All the lists

In our Streamlit app, we used the following names for the CSVs produced for each attribute's potential options:
* Genre: `genreLabels_list.csv`
* Nationality: `nationalities_list.csv`
* Work: `works_list.csv`
* Composer: `composers_list.csv`